In [1]:
import os
import numpy as np
import cv2 as cv
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.svm import SVC

# Traditional Features Construction

In [ ]:
def feature_extraction(image):
    img = cv.imread(image)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    sift = cv.SIFT_create()
    kp, des = sift.detectAndCompute(gray, None)
    img = cv.drawKeypoints(gray, kp, img)
    cv.imwrite('sift_keypoints.jpg', img)

In [ ]:
def k_means_clustering(X):
    kmeans = KMeans(n_clusters=100, random_state=0, init="random", n_init="auto").fit(X)
    
    return kmeans

In [ ]:
def dimension_reduction(features):
    pca = PCA(n_components=2)
    transformed_data = pca.fit_transform(features)
    
    for i in range(4):
        c = np.flatnonzero(class_labels == i)
        plt.scatter(transformed_data[c, 0], transformed_data[c, 1])
    plt.show()
    
    return transformed_data